In [1]:
from openai import OpenAI
import requests
import base64
import json
import os

client = OpenAI(
    api_key="sk-rSO99Pa2js_qptjzWZoDUg",
    base_url="https://api.thucchien.ai"
)

API_KEY = "sk-rSO99Pa2js_qptjzWZoDUg"
GEMINI_URL = "https://api.thucchien.ai/gemini/v1beta/models/gemini-2.5-flash-image-preview:generateContent"

def load_history():
    if os.path.exists("image_history.json"):
        with open("image_history.json", "r", encoding="utf-8") as f:
            return json.load(f)
    return []

def save_history(hist):
    with open("image_history.json", "w", encoding="utf-8") as f:
        json.dump(hist, f, ensure_ascii=False, indent=2)

In [ ]:
history = load_history()
print("Imagenbot san sang! (go 'exit' de thoat)\n")

while True:
    print("\nChon che do:")
    print("1. Sinh anh tu mo ta text")
    print("2. Sinh anh moi tu anh co san + mo ta")
    mode = input("Nhap lua chon (1 hoac 2): ").strip()

    if mode.lower() in ["exit", "quit", "bye"]:
        save_history(history)
        print("Tam biet!")
        break

    if mode == "1":
        prompt = input("\nNhap mo ta anh: ").strip()
        if not prompt:
            print("Ban chua nhap mo ta.")
            continue

        response = client.images.generate(
            model="gemini-2.5-flash-image-preview",
            prompt=prompt,
            n="1",
            extra_body={
                "aspect_ratio": "1:1"
            }
        )

        b64_data = response.data[0].b64_json
        image_data = base64.b64decode(b64_data)
        output_file = "generated_image.png"
        with open(output_file, "wb") as f:
            f.write(image_data)
        print("Anh da duoc tao tu mo ta:", prompt)
        print("Da luu tai:", output_file)

        history.append({"type": "text_to_image", "prompt": prompt, "output": output_file})

    elif mode == "2":
        image_path = input("\nNhap duong dan anh goc (VD: input.jpg): ").strip()
        if not os.path.exists(image_path):
            print("Khong tim thay file anh.")
            continue

        edit_prompt = input("Nhap mo ta chinh sua anh: ").strip()
        if not edit_prompt:
            print("Ban chua nhap mo ta.")
            continue

        try:
            # Đọc ảnh và encode base64
            with open(image_path, "rb") as f:
                image_b64 = base64.b64encode(f.read()).decode("utf-8")

            # Payload gửi đến Gemini (text + image input → image output)
            payload = {
                "contents": [
                    {
                        "parts": [
                            {
                                "text": (
                                    f"Here is an image. Please generate a new version "
                                    f"based on this image with the following modification: {edit_prompt}. "
                                    f"The new image should reflect this change realistically."
                                )
                            },
                            {
                                "inline_data": {
                                    "mime_type": "image/png",
                                    "data": image_b64
                                }
                            }
                        ]
                    }
                ],
                "generationConfig": {
                    "imageConfig": {
                        "aspectRatio": "1:1"
                    }
                }
            }

            headers = {
                "x-goog-api-key": API_KEY,
                "Content-Type": "application/json"
            }

            response = requests.post(GEMINI_URL, headers=headers, json=payload)
            data = response.json()

            if response.status_code != 200:
                print("Loi API:", response.status_code, data)
                continue

            try:
                # Gemini có thể trả inline_data hoặc inlineData (chữ hoa/thường khác nhau)
                parts = data["candidates"][0]["content"]["parts"]
                img_b64 = None
                for p in parts:
                    if "inline_data" in p:
                        img_b64 = p["inline_data"]["data"]
                    elif "inlineData" in p:
                        img_b64 = p["inlineData"]["data"]

                if not img_b64:
                    print("Gemini khong tra ve du lieu anh. Phan hoi API:")
                    print(json.dumps(data, indent=2))
                    continue

                img = base64.b64decode(img_b64)
                output_file = "edited_image.png"
                with open(output_file, "wb") as f:
                    f.write(img)

                print("Anh moi da duoc tao theo mo ta:", edit_prompt)
                print("Da luu tai:", output_file)

                history.append({
                    "type": "image_to_image",
                    "input_image": image_path,
                    "prompt": edit_prompt,
                    "output": output_file
                })

            except Exception as e:
                print("Loi khi giai ma anh:", e)
                print(json.dumps(data, indent=2))

        except Exception as e:
            print("Loi:", e)
            if "response" in locals():
                print("Phan hoi API:", response.text)

    else:
        print("Vui long chon 1 hoac 2.")

save_history(history)